# WEB幹事スクレイピング（基礎編）

In [1]:
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
"""
pandas==1.3.3
beautifulsoup4==4.10.0
webdriver-manager==3.5.1
requests
tqdm
"""

'\npandas==1.3.3\nbeautifulsoup4==4.10.0\nwebdriver-manager==3.5.1\nrequests\ntqdm\n'

In [3]:
options = Options()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/takada/.wdm/drivers/chromedriver/mac64/95.0.4638.54/chromedriver] found in cache


In [4]:
#driver.quit()

## 1. 検索結果1ページ目から個別ページのリンクを取得する

In [5]:
url = 'https://web-kanji.com/search/akita'
driver.get(url)

In [6]:
company_items = driver.find_elements(By.CSS_SELECTOR, '.companies-item')

In [7]:
print(len(company_items))
print(company_items[0].get_attribute('href'))

20
https://web-kanji.com/companies/kanata-factory


In [8]:
single_company_urls = [item.get_attribute('href') for item in company_items]

### 関数化

In [9]:
def fetch_single_company_urls(driver, url):
    """
    パラメータ:
        driver: WebDriver
        url: 検索結果の一覧ページ
    返却値:
        single_company_urls: 制作会社個別ページのURLリスト

    urlに載っている制作会社個別ページのURLを取得する
    """
    driver.get(url)
    sleep(1)
    company_items = driver.find_elements(By.CSS_SELECTOR, '.companies-item')
    single_company_urls = [item.get_attribute('href') for item in company_items]
    return single_company_urls


## 2. 個別ページの会社概要の情報を抽出する

In [10]:
url = single_company_urls[0]
driver.get(url)

In [11]:
#dl_element = driver.find_elements(By.CSS_SELECTOR, '.company-data.is-narrow')[0]
dl_element = driver.find_element(By.CSS_SELECTOR, '.company-data.is-narrow')
dt_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dt')
dd_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dd')

datum = {}
for dt, dd in zip(dt_elements, dd_elements):
    datum[dt.text] = dd.text.replace('\n', ' ')
datum

{'会社名': '株式会社Kanata factory',
 '設立': '2020年10月1日',
 '代表': '金沢 佑紀',
 '本社所在地': '〒 016-0831 秋田県能代市元町4-11',
 '資本金': '300万円',
 '社員数': '3人',
 'URL': 'https://kanata-factory.co.jp/'}

### 関数化

In [12]:
def extract_company_data(driver, url):
    """
    パラメータ:
        driver: WebDriver
        url: 個別ページURL
    返却値:
        data: 制作会社の会社情報のdict
              (例) 
              {
                  'WEB幹事_URL': 'http://web-kanji....', 
                  '会社名': 'xxx株式会社', 
                  '代表': '山田 太郎', 
                  'URL': 'http://xxx.co.jp/', 
                  ...
              }
    """
    driver.get(url)
    sleep(1)
    dl_element = driver.find_element(By.CSS_SELECTOR, '.company-data.is-narrow')
    dt_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dt')
    dd_elements = dl_element.find_elements(By.CSS_SELECTOR, 'dd')

    data = {}
    data['WEB幹事_URL'] = url
    for dt, dd in zip(dt_elements, dd_elements):
        data[dt.text] = dd.text.replace('\n', ' ')
    return data

## 3. 20社の会社概要を取得する ※ 今回は3社

ただし、今回は同時に20件アクセスするとサーバに負荷をかけてしまうので3件とします。

In [13]:
search_url = 'https://web-kanji.com/search/akita'
single_company_urls = fetch_single_company_urls(driver, search_url)

In [14]:
# 3社のみにフィルタ
print(len(single_company_urls))
single_company_urls = single_company_urls[:3]
print(len(single_company_urls))

20
3


In [15]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]

In [16]:
df = pd.DataFrame(company_data).fillna('')
df

,WEB幹事_URL,会社名,設立,代表,本社所在地,資本金,社員数,URL
0,https://web-kanji.com/companies/kanata-factory,株式会社Kanata factory,2020年10月1日,金沢 佑紀,〒 016-0831 秋田県能代市元町4-11,300万円,3人,https://kanata-factory.co.jp/
1,https://web-kanji.com/companies/crossfade,合資会社クロスフェイド,2001年10月,,〒 015-0041 秋田県由利本荘市薬師堂字境田54-101,,,http://www.crossfade.jp/
2,https://web-kanji.com/companies/kokua-d,コクア・デザイン・オフィス,2004年11月,齋藤 学,〒 010-1502 秋田県秋田市下浜長浜字長坂159−6,100万円,1,http://www.kokua-d.com/


## 4. 検索結果の2ページ目以降に掲載されている制作会社個別ページも取得する

### 方法1 (易しい、手作業多い)

検索結果のページネーションURLを直書きで指定する

In [17]:
search_urls = [
    'https://web-kanji.com/search/akita',
    'https://web-kanji.com/search/akita/page/2',
]

In [18]:
single_company_urls = []
for url in search_urls:
    urls = fetch_single_company_urls(driver, url)
    single_company_urls.extend(urls)

print(len(single_company_urls))
single_company_urls = single_company_urls[:3]
print(len(single_company_urls))

36
3


In [19]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]
df = pd.DataFrame(company_data).fillna('')
df

,WEB幹事_URL,会社名,設立,代表,本社所在地,資本金,社員数,URL
0,https://web-kanji.com/companies/kanata-factory,株式会社Kanata factory,2020年10月1日,金沢 佑紀,〒 016-0831 秋田県能代市元町4-11,300万円,3人,https://kanata-factory.co.jp/
1,https://web-kanji.com/companies/crossfade,合資会社クロスフェイド,2001年10月,,〒 015-0041 秋田県由利本荘市薬師堂字境田54-101,,,http://www.crossfade.jp/
2,https://web-kanji.com/companies/kokua-d,コクア・デザイン・オフィス,2004年11月,齋藤 学,〒 010-1502 秋田県秋田市下浜長浜字長坂159−6,100万円,1,http://www.kokua-d.com/


### 方法2 (難しい、手作業少ない)

In [20]:
search_url = 'https://web-kanji.com/search/akita'
driver.get(search_url)

In [21]:
search_urls = []
current_url = search_url
while True:
    driver.get(current_url)
    sleep(1)
    search_urls.append(current_url)
    try:
        next_a = driver.find_element(By.CSS_SELECTOR, '.pagination-item > a[rel="next"]')
    except NoSuchElementException:
        break
    current_url = next_a.get_attribute('href')

### 関数化

In [22]:
def fetch_search_pagination_urls(driver, search_url):
    """
    パラメータ:
        driver: WebDriver
        search_url: 検索結果１ページ目のURL
    返却値:
        search_urls: 検索条件のページネーションURLリスト
    
    検索結果のページネーションを自動で検知してページネーションのURLをすべて取得する
    """
    search_urls = []
    current_url = search_url
    while True:
        driver.get(current_url)
        sleep(1)
        search_urls.append(current_url)
        try:
            next_a = driver.find_element(By.CSS_SELECTOR, '.pagination-item > a[rel="next"]')
        except NoSuchElementException:
            break
        current_url = next_a.get_attribute('href')
    return search_urls

In [23]:
search_urls = fetch_search_pagination_urls(driver, search_url)
search_urls

['https://web-kanji.com/search/akita',
 'https://web-kanji.com/search/akita/page/2']

In [24]:
single_company_urls = []
for url in search_urls:
    urls = fetch_single_company_urls(driver, url)
    single_company_urls.extend(urls)
print(len(single_company_urls))
single_company_urls = single_company_urls[:3]

36


In [25]:
company_data = [extract_company_data(driver, url) for url in single_company_urls]
df = pd.DataFrame(company_data).fillna('')
df

,WEB幹事_URL,会社名,設立,代表,本社所在地,資本金,社員数,URL
0,https://web-kanji.com/companies/kanata-factory,株式会社Kanata factory,2020年10月1日,金沢 佑紀,〒 016-0831 秋田県能代市元町4-11,300万円,3人,https://kanata-factory.co.jp/
1,https://web-kanji.com/companies/crossfade,合資会社クロスフェイド,2001年10月,,〒 015-0041 秋田県由利本荘市薬師堂字境田54-101,,,http://www.crossfade.jp/
2,https://web-kanji.com/companies/kokua-d,コクア・デザイン・オフィス,2004年11月,齋藤 学,〒 010-1502 秋田県秋田市下浜長浜字長坂159−6,100万円,1,http://www.kokua-d.com/


## 5. 抽出したデータをファイルに書き出す

### tsvフォーマットで書き出す

In [26]:
df.to_csv('company_data.tsv', sep='\t')

In [28]:
!pip freeze | grep webdri

webdriver-manager==3.5.1


In [29]:
try:
    3 / 0
except ZeroDivisionError:
    print('0で割ってるよ')

0で割ってるよ
